# Importing the libraries

In [ ]:
#Importing libraries
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
import os.path as osp
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive

# Reading and oragnizing data

In [ ]:
#Read csv file
df0 = pd.read_csv('Colab Notebooks/data/final-15oct.csv')
test = pd.read_csv('Colab Notebooks/data/test_walk1.csv')
df0.head()

In [ ]:
df0.info()

In [ ]:
#Deal with empty values in train
def clean(df0):
  if((df0.isnull().any()).any()):
    print("Null values found")
    print(df.isnull().sum())
    df0.replace('',np.nan, inplace=True)
    df0=df0.dropna()
    print("Null droppped")
  else:
    print('No Null values')
  return df0
df0 = clean(df0)

In [ ]:
#visuaise records
val=df0['Activity'].value_counts()
print(val)
print('-------------------------------------')
plt.figure(figsize=(15,5))
val.plot(kind='bar')

In [ ]:
#Split dataset into X and y
def split(df):
  X=df.iloc[:,1:4]
  y=df.iloc[:,4:5]
  X= X.astype(float)
  X= np.asarray(X)
  return X,y

#One Hot Encode Y
def ohe(y):
  y=pd.get_dummies(y)
  y= y.astype(int)
  y= np.asarray(y)
  return y

#Inverse One Hot Encoding
def de_ohe(y):
  deohe = np.array([])
  for i in y:
    n = np.argmax(i)
    if n == 3:
      deohe = np.append(deohe, ['WALKING'])
    elif n == 2:
      deohe = np.append(deohe, ['UPSTAIRS'])
    elif n == 1:
      deohe = np.append(deohe, ['SITTING'])
    elif n == 0:
      deohe = np.append(deohe, ['DOWNSTAIRS'])
  return deohe

#Standard Scale X
def scale(X):
  sc=StandardScaler()
  X=sc.fit_transform(X)
  return X

#Normalise X
def mmscale(X):
  mm = MinMaxScaler()
  mm.fit_transform(X)
  return X

#Makes no. samples per class equal
def symmetrify(df):
  unique = []
  df2 = pd.DataFrame()
  val = df['Activity'].value_counts()
  count = list(val)
  mini = min(count)
  for i in range(0,len(df)):
    act = df.at[i, 'Activity']
    if act not in unique:
      unique = unique + [act]
      df2 = df2.append(df.iloc[i : (i+mini), :])
  print(unique)
  print(df2['Activity'].value_counts())
  return df2, mini

#Create batches of data
def temporalize(ts, X, y):
    X_in=[]
    y_in=[]
    for i in range(ts, len(X), ts):
        n1 = X[i-ts:i,:]
        X_in.append(n1.tolist())
        if type(y) != int:
            n2=y[i-ts:i,:]
            y_in.append(n2.tolist())
    X_in=np.asarray(X_in ,dtype=np.float32)
    if type(y) != int:
        y_in=np.asarray(y_in, dtype=np.float32)
    return X_in, y_in

#Visualize history object
def visualize(history):
  plt.figure(figsize=(10,10))
  plt.subplot(2, 1, 1)
  plt.plot(history.history['loss'])
  plt.xlabel('EPOCH', color='w')
  plt.ylabel('LOSS' , color='w')
  plt.subplot(2, 1, 2)
  plt.plot(history.history['acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  return 0

In [ ]:
df, mini = symmetrify(df0)

In [ ]:
#plot of downstairs
a = 0
b = 500
fig, axes = plt.subplots(2, 2, figsize=(22, 8))
ts = df["TS"].iloc[0:500]
z= df["Z"].iloc[0:500]
y= df["Y"].iloc[0:500]
x= df["X"].iloc[0:500]
axes[0,0].plot(ts, x, lw=1)
axes[0,0].plot(ts, y, lw=1)
axes[0,0].plot(ts, z, lw=1)
axes[0,0].legend(['x', 'y', 'z'], loc='upper left')
axes[0,0].set_title("Downstairs")

#plot of walking
a = mini + 1
b = mini + 501
ts = df["TS"].iloc[a:b]
z= df["Z"].iloc[a:b]
y= df["Y"].iloc[a:b]
x= df["X"].iloc[a:b]
axes[0,1].plot(ts, x, lw=1)
axes[0,1].plot(ts, y, lw=1)
axes[0,1].plot(ts, z, lw=1)
axes[0,1].legend(['x', 'y', 'z'], loc='upper left')
axes[0,1].set_title("Walking")

#plot of upstairs
a = mini*2 + 1
b = mini*2 + 501
ts = df["TS"].iloc[a:b]
z= df["Z"].iloc[a:b]
y= df["Y"].iloc[a:b]
x= df["X"].iloc[a:b]
axes[1,0].plot(ts, x, lw=1)
axes[1,0].plot(ts, y, lw=1)
axes[1,0].plot(ts, z, lw=1)
axes[1,0].legend(['x', 'y', 'z'], loc='upper left')
axes[1,0].set_title("Upstairs")

#plot of sitting
a = mini*3 + 1
b = mini*3 + 501
ts = df["TS"].iloc[a:b]
z= df["Z"].iloc[a:b]
y= df["Y"].iloc[a:b]
x= df["X"].iloc[a:b]
axes[1,1].plot(ts, x, lw=1)
axes[1,1].plot(ts, y, lw=1)
axes[1,1].plot(ts, z, lw=1)
axes[1,1].legend(['x', 'y', 'z'], loc='upper left')
axes[1,1].set_title("Sitting")
plt.show()

In [ ]:
X,y = split(df)

X_mm = mmscale(X)

y_ohe = ohe(y)

In [ ]:
def temporalizetry(ts, X, y):
    X_in=[]
    y_in=[]
    for i in range(ts, len(X) - ts, 20):
        xs = X[i:i+ts,0:1]
        ys = X[i:i+ts,1:2]
        zs = X[i:i+ts,2:3]
        X_in.append([np.asarray(xs),np.asarray(ys),np.asarray(zs)])
        y_in.append(y[i:i+ts,:].tolist())
    X_in=np.asarray(X_in ,dtype=np.float32)
    y_in=np.asarray(y_in, dtype = np.float32)
    return X_in, y_in

In [ ]:
X_in, y_in=temporalizetry(70, X, y_ohe)
X_in = X_in.reshape(8043,70,3)

In [ ]:
np.array(X_in).shape

In [ ]:
X_in[0][0]

# Assemble the model

In [ ]:
net = ''
from datetime import datetime 
from tensorflow import keras
start_time = time.time()
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

net= Sequential()

adam = Adam(lr=0.01)

#net.add(LSTM(units=100, input_shape=(X_in.shape[1],X_in.shape[2]),return_sequences=True))
net.add(LSTM(units=64, input_shape=(X_in.shape[1],X_in.shape[2]),return_sequences=True))
#net.add(Dropout(0.2))
net.add(LSTM(units=64, return_sequences=True))
#net.add(Dense(units=64))

#net.add(Dropout(0.2))

net.add(Dense(units=4, activation='softmax'))

net.compile(optimizer=adam, loss= 'categorical_crossentropy', metrics=['accuracy'])

history=net.fit(X_in, y_in, shuffle=False, verbose=1, epochs=2, batch_size=1024, callbacks=[tensorboard_callback])
print("----------- Metrics -----------")

print("Accuracy:" + str(max(history.history['acc'])))

print("Loss:" + str(max(history.history['loss'])))

visualize(history)

print("----------- Time: %d minutes -----------" % round((time.time() - start_time)/60,2))

In [ ]:
test_X, test_y_eval = split(test)
test_y = ohe(test_y_eval)
test_X = mmscale(test_X)
test_X, test_y=temporalize(70, test_X, test_y)
test_y_pred=net.predict(test_X)
rows = test_y_pred.shape[0]*test_y_pred.shape[1]
test_y_pred= test_y_pred.reshape(rows, 4)
test_y_pred = (de_ohe(test_y_pred))
print(test_y.shape)
print(test_y_pred.shape)

In [ ]:
print(classification_report(test_y_eval.iloc[0 : rows], test_y_pred))